In [219]:
import pandas as pd
import re  

In [220]:
# Daten laden
bookings = pd.read_csv( 'data/raw/crossbooking_challenge.csv' )

In [221]:
bookings.head()

,turnover_id,payee_payer_name,holder_name
0,b892f151-e54d-4cdc-ad3b-bb49c6f10197,Stefan und Ulrike Steffens,Stefan und Manuela Steffens
1,85921cb1-9d3c-4ee0-aefa-5b758eef8701,Drk Krejsverband Claire Siege V,Maren Hecker
2,63cb68a0-8a04-11ec-1d14-82bb5829b41b,Emslaendische Volksbank Eg,Hinrich Schleich
3,fe383450-bc4e-11ec-c954-6c39d3c8dc12,Mig Gmbh Und Co Fonds6 Kg,Käthe Finke
4,9a372a30-e9e2-11ec-e558-8689140089a8,Bundesagentur Rosel Arbeit Service Lehmann,Käthe Finke


In [222]:
def comma_splitting( x ):
    """
    Namen richtig drehen.
    """
    if type( x ) != str:
        return None
    splitted = x.split( ',' )
    if len(splitted) == 2:
        return f'{splitted[1].strip()} {splitted[0].strip()}'
    else:
        return x

In [223]:
def replace_special_chars( x ):
    """
    Macht natürlich nicht immer Sinn. In den Beispielen vllt schon.
    """
    if type( x ) != str:
        return None
    res = x 
    res = res.replace('ü','ue')
    res = res.replace('ö','oe')
    res = res.replace('ä','ae')
    res = res.replace('.','')
    return res

In [224]:
def replace_title( x ):
    """
    Kann natürlich noch viel mehr geben. Kann auch im String klassifizieren mit NLP Modellen. 
    """
    if type( x ) != str:
        return None
    res = x 
    res = re.sub( r'(^|\W)herr(^|\W)', '', res )
    res = re.sub( r'(^|\W)dr(^|\W)', '', res )
    res = re.sub( r'(^|\W)prof(^|\W)', '', res )
    res = res.replace('dr ','')
    return res

In [225]:
def simplify_names( x ): 
    """
    Vermutlich Fehler bei mehr Daten, aber macht hier bei ein paar Beispielen Sinn.
    """
    if type( x ) != str:
        return None
    splitted = x.split( ' ' )
    if len(splitted) >= 2 and not ('und' in splitted or 'oder' in splitted):
        return f'{splitted[0].strip()}{splitted[-1].strip()}'
    else:
        return x

In [226]:
def joint_splitting( x ):
    """
    Gemeinschaftskonten splitten. Sollte wohl noch um andere Varianten erweitert werden. 
    """
    if type( x ) != str:
        return pd.Series([None, None])
    splitted = x.split( ' und ' )
    if len(splitted) == 2:
        splitted_last = splitted[1].strip().split(' ')
        surname = splitted_last[-1].strip()
        forename_first = splitted[0].strip()
        forename_second = ' '.join( splitted_last[:-1] )
        return pd.Series([f'{forename_first} {surname}', f'{forename_second} {surname}'])
    else:
        return pd.Series([None, None])

In [227]:
# Alle Funktionen einmal zum Preprocessen anweden
def preprocess(df, column):
    df[column] = df[column].str.lower() # lower case for matching
    df[column] = df[column].apply( replace_special_chars ) # replace special chars
    df[column] = df[column].apply( replace_title ) # replace special chars
    df[column] = df[column].apply( comma_splitting ) # split , separated names
    df[f'{column}_short'] = df[column].apply( simplify_names ) # shorten names, e.g., hans peter müller => hans müller
    splitted = [f'{column}_a', f'{column}_b']
    df[splitted] = df[column].apply( joint_splitting ) # split joint accounts
    return df

In [228]:
# für beide Spalten
test = preprocess( bookings, 'payee_payer_name' )
test = preprocess( test, 'holder_name' )

In [229]:
# Direkt A => A
test['rule_1_direct'] = test['payee_payer_name'] == test['holder_name']
test['rule_1_direct_unsafe'] = test['payee_payer_name_short'] == test['holder_name_short']

In [230]:
# Indirekt A+B => A | B
first_match = test['payee_payer_name'] == test['holder_name_a']
second_match = test['payee_payer_name'] == test['holder_name_b']
test['rule_2_joint_to_single'] =  first_match | second_match

In [231]:
# Indirekt A | B => A+B
first_match = test['payee_payer_name_a'] == test['holder_name']
second_match = test['payee_payer_name_b'] == test['holder_name']
test['rule_3_single_to_joint'] =  first_match | second_match

In [232]:
# Eine Regel sollte passen
rules = [ 'rule_1_direct', 'rule_2_joint_to_single', 'rule_3_single_to_joint', 'rule_1_direct_unsafe' ]
test['crossbooking'] = False
for rule in rules: 
    test['crossbooking'] = test['crossbooking'] | test[rule]

In [233]:
# Welche Regel trifft?
test['rule_1_direct'].sum(), test['rule_1_direct_unsafe'].sum(), test['rule_2_joint_to_single'].sum(), test['rule_3_single_to_joint'].sum(), test['crossbooking'].sum()

(60, 64, 0, 2, 66)

In [234]:
test[['turnover_id','payee_payer_name','holder_name','crossbooking']].to_csv( 'data/output/test.csv', sep=';' )

## Kommentare
* Hat Stefan zwei Frauen? Soll sowas trotzdem zählen?
* Generell würd ich hier mit Blick auf mehr Daten
    * die Regeln verifizieren/testen und eher sicher festlegen
    * zusätzlich noch fuzzy matchen und Randfälle beobachten zur Verfeinerung
* Mapping von A+B => A auf A => B könnte man sicher noch machen, kommt aber hier vermutlich nicht vor!?